In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
import time
from tqdm.notebook import tqdm
from glob import glob
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings
import numpy as np
import datetime
import re

/tmp/ipykernel_17845/1883619435.py:17: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
url_main = "https://www.applyhome.co.kr/ai/aia/selectAPTLttotPblancListView.do"

# Chrome 드라이버 초기화
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.get(url_main)

In [3]:
# Selenium을 사용하여 현재 페이지의 소스 가져오기
present_url = driver.page_source

In [4]:
# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(present_url, 'html.parser')

### 철근 누락 적발 시공사 명단

In [5]:
#https://realty.chosun.com/site/data/html_dir/2023/07/31/2023073101968.html, 국토교통부 재인용 기사
bad_company = ["대보건설", "대림(DL)건설", "삼환기업", "이수건설", "한신건설", "양우종합건설", "효성중공업", "대우산업개발", "에이스건설", "한신공영", "태평양개발", "동문건설", "남영건설", "한라"]

### 청약 시공사 리스트 출력

In [20]:
# 데이터를 담을 리스트 초기화
data = []

In [15]:
# tr 요소들을 찾아 반복
for tr in soup.find_all('tr'):
    # 각 컬럼의 값을 가져와서 딕셔너리로 저장
    td_element = tr.find('td')
    b_element = tr.find('b')
    word_cut_element = tr.find(class_='word_cut')

    # 요소가 존재하는지 확인하고 데이터에 추가
    if td_element and b_element and word_cut_element:
        region = td_element.text.strip()
        housing_name = b_element.text.strip()
        construction_company = word_cut_element.text.strip()

        # 딕셔너리에 저장
        row = {'지역': region, '주택명': housing_name, '시공사': construction_company}
        data.append(row)

# 중복된 데이터 제거
data = [dict(t) for t in {tuple(d.items()) for d in data}]

# 데이터 프레임 생성
df = pd.DataFrame(data)

In [21]:
# 결과 출력
print(df)

NameError: name 'df' is not defined

### 페이지 이동 추가

In [19]:
# 데이터를 담을 리스트 초기화
data = []

# 다음 페이지 버튼을 클릭하여 더 많은 데이터를 로드하고 파싱합니다.
while True:
    # 현재 페이지의 HTML 가져오기
    present_url = driver.page_source
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(present_url, 'html.parser')
    
    # tr 요소들을 찾아 반복
    for tr in soup.find_all('tr'):
        try:
            # 각 컬럼의 값을 가져와서 딕셔너리로 저장
            td_element = tr.find('td')
            b_element = tr.find('b')
            word_cut_element = tr.find(class_='word_cut')

            # 요소가 존재하는지 확인하고 데이터에 추가
            if td_element and b_element and word_cut_element:
                region = td_element.text.strip()
                housing_name = b_element.text.strip()
                construction_company = word_cut_element.text.strip()

                # 딕셔너리에 저장
                row = {'지역': region, '주택명': housing_name, '시공사': construction_company}
                data.append(row)
        except AttributeError as e:
            print("Error:", e)
            continue
    
    # 현재 페이지 번호 가져오기
    current_page_element = soup.find('a', class_='active')
    current_page = int(current_page_element.text.strip())

    # 다음 페이지로 이동
    next_page = current_page + 1
    try:
        next_page_button = driver.find_element(By.XPATH, f'//a[@href="?pageIndex={next_page}"]')
        next_page_button.click()
        time.sleep(2)  # 페이지가 로드될 때까지 잠시 대기
    except NoSuchElementException:
        break
    
# 데이터 프레임 생성
df_APT_list = pd.DataFrame(data)
next_page = 1

In [20]:
# 결과 출력
print(df_APT_list)
df_APT_list.to_excel("../data/APT_list.xlsx", engine='xlsxwriter')

     지역                   주택명         시공사
0    경기     남양주별내Ａ１－２　행복주택　추가     대보건설(주)
1    경기       이천마장　Ａ３　행복주택　추가  한국토지주택공사서울
2    경기      안성공도　참아름　국민임대　예비     주공 안성공도
3    광주  광주효천２　Ａ３　１０년　분납임대　예비  한국토지주택광주전남
4    광주  광주효천２　Ａ１　１０년　공공임대　예비  한국토지주택광주전남
..   ..                   ...         ...
109  경기  위례신도시　Ａ３－４ｂＢＬ　우미린　１차     (주)우미개발
110  부산      힐스테이트　명륜　２차　（부산）  현대엔지니어링(주)
111  강원          태백소도　국민임대　예비  한국토지주택공사강원
112  강원         동해천곡６　국민임대　예비  한국토지주택공사강원
113  전북         디오션시티　더샵　（군산）    (주)포스코건설

[114 rows x 3 columns]


In [25]:
# "시공사" 컬럼의 텍스트가 없는 행 제외
df_APT_list = df_APT_list[df_APT_list['시공사'].notna()]

# bad_company를 데이터프레임으로 변환
bad_companys = pd.DataFrame({'bad_company': bad_company})

# "시공사" 컬럼의 요소와 텍스트가 같거나 서로 포함되는 문자열 관계인 경우 찾기
bad_company_elements = []
for index, row in df_APT_list.iterrows():
    for company in bad_company:
        if company in row['시공사'] or row['시공사'] in company:
            bad_company_elements.append(row)
            break

# bad_company_elements를 데이터프레임으로 변환
bad_companys = pd.DataFrame(bad_company_elements)


In [26]:
# 결과 출력
print(bad_companys)

     지역                   주택명      시공사
0    경기     남양주별내Ａ１－２　행복주택　추가  대보건설(주)
6    충남   논산내동２－１　１０년　공공임대　예비  한신공영(주)
34   세종     세종　더휴　예미지　Ｌ２　（민영）  한신공영(주)
35   세종     세종　더휴　예미지　Ｌ２　（국민）  한신공영(주)
36   세종     세종　더휴　예미지　Ｌ１　（민영）  한신공영(주)
37   세종     세종　더휴　예미지　Ｌ１　（국민）  한신공영(주)
106  경기  김포한강Ａｃ－０５　１０년　공공임대예비         


### 테그 사이에 정보가 없으면 -> None! space (" ")와는 다름!

In [27]:
# "시공사" 컬럼의 요소와 텍스트가 같거나 서로 포함되는 문자열 관계인 경우 찾기
bad_company_elements = []
for index, row in df_APT_list.iterrows():
    for company in bad_company:
        # row['시공사'] 또는 company가 None이 아니고, 서로 포함되는 문자열 관계인 경우
        if row['시공사'] and company and (company in row['시공사'] or row['시공사'] in company):
            bad_company_elements.append(row)
            break

# None을 제외한 bad_company_elements를 데이터프레임으로 변환
bad_companys = pd.DataFrame([elem for elem in bad_company_elements if elem is not None])

# bad_company_elements를 데이터프레임으로 변환
bad_companys = pd.DataFrame(bad_company_elements)

In [28]:
# 결과 출력
print(bad_companys)

    지역                  주택명      시공사
0   경기    남양주별내Ａ１－２　행복주택　추가  대보건설(주)
6   충남  논산내동２－１　１０년　공공임대　예비  한신공영(주)
34  세종    세종　더휴　예미지　Ｌ２　（민영）  한신공영(주)
35  세종    세종　더휴　예미지　Ｌ２　（국민）  한신공영(주)
36  세종    세종　더휴　예미지　Ｌ１　（민영）  한신공영(주)
37  세종    세종　더휴　예미지　Ｌ１　（국민）  한신공영(주)


In [29]:
# 새로운 인덱스 설정
bad_companys.reset_index(drop=True, inplace=True)
print(bad_companys)

   지역                  주택명      시공사
0  경기    남양주별내Ａ１－２　행복주택　추가  대보건설(주)
1  충남  논산내동２－１　１０년　공공임대　예비  한신공영(주)
2  세종    세종　더휴　예미지　Ｌ２　（민영）  한신공영(주)
3  세종    세종　더휴　예미지　Ｌ２　（국민）  한신공영(주)
4  세종    세종　더휴　예미지　Ｌ１　（민영）  한신공영(주)
5  세종    세종　더휴　예미지　Ｌ１　（국민）  한신공영(주)


### 지역, 주택명, 시공사 수집하는 함수

In [30]:
def parse_data(driver):
    # 데이터를 담을 리스트 초기화
    data = []

    # 다음 페이지 버튼을 클릭하여 더 많은 데이터를 로드하고 파싱합니다.
    while True:
        # 현재 페이지의 HTML 가져오기
        present_url = driver.page_source
        
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(present_url, 'html.parser')
        
        # tr 요소들을 찾아 반복
        for tr in soup.find_all('tr'):
            try:
                # 각 컬럼의 값을 가져와서 딕셔너리로 저장
                td_element = tr.find('td')
                b_element = tr.find('b')
                word_cut_element = tr.find(class_='word_cut')

                # 요소가 존재하는지 확인하고 데이터에 추가
                if td_element and b_element and word_cut_element:
                    region = td_element.text.strip()
                    housing_name = b_element.text.strip()
                    construction_company = word_cut_element.text.strip()

                    # 딕셔너리에 저장
                    row = {'지역': region, '주택명': housing_name, '시공사': construction_company}
                    data.append(row)
            except AttributeError as e:
                print("Error:", e)
                continue
        
        # 현재 페이지 번호 가져오기
        current_page_element = soup.find('a', class_='active')
        current_page = int(current_page_element.text.strip())

        # 다음 페이지로 이동
        next_page = current_page + 1
        try:
            next_page_button = driver.find_element(By.XPATH, f'//a[@href="?pageIndex={next_page}"]')
            next_page_button.click()
            time.sleep(2)  # 페이지가 로드될 때까지 잠시 대기
        except NoSuchElementException:
            break
    
    # 데이터 프레임 생성
    df_APT_list = pd.DataFrame(data)
    
    return df_APT_list

### 철근 누락 건설사 여부 확인

In [31]:
def check_relationship(df, bad_company):
    # "시공사" 컬럼의 요소와 텍스트가 같거나 서로 포함되는 문자열 관계인 경우 찾기
    bad_company_elements = []
    for index, row in df.iterrows():
        for company in bad_company:
            # row['시공사'] 또는 company가 None이 아니고, 서로 포함되는 문자열 관계인 경우
            if row['시공사'] and company and (company in row['시공사'] or row['시공사'] in company):
                bad_company_elements.append(row)
                break

    # None을 제외한 bad_company_elements를 데이터프레임으로 변환
    bad_companies = pd.DataFrame([elem for elem in bad_company_elements if elem is not None])

    return bad_companies

### 5개년 조사